# **DDL - Creating & Altering - Schemas & Tables**

In this first portion of code we are going to create Schemas, these will combine and organize the future tables into two distinct types of data: 

1. **Dimensions (dim)**
2. **Facts (f)**

Code Breakdown:

- First portion is code is making sure that the correct Database is being used; this is not required however because it is such an easy addition, I personally will add this to all my scripts.
- Second is making sure that these schemas haven't already been created into the database; and if they are, no futher processes will be complete.
- Lastly this is the creation of the schemas; the facts schema being shortened to 'f' & Dimensions Schemas being shortened to 'dim'.

In [5]:
USE RHNSCC_DD;
GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006

### **1.** The next portion of code is the creation of the two **Dimansion Tables**: **dim.Products** and **dim.People**.

Code Breakdown:

- Similar to any created query first I start with USE Database so that I can confirm that the created tables are being added to the correct DB.
    
- Next Code up is the IF NOT EXISTS code which confirms that prior to the creation of the Table, the designated DB does not have a Table named Products in the dim Schema. While this isn't required, it does allow for the query to be executed while not giving errors, essentially making not being done if there is Table in the DB already.
    
- The Create Table code is then added in. This section includes:
    
    - the Schema & Table name you are requesting (i.e. dim.Products) and
    - The row titles. Within this you will also include:
        - The Data Type, the length of units (50) as well as if the data is either required or not required (NULL or NOT NULL)
- The final portion of the Code is an alteration of the Code. While this can be completed within the CREATE TABLE Code, it's more recommended to complete it via ALTER TABLE code as you can easily create a distintive name for the Constraint.
    
    - Adding a Key while creating the table makes it a bit more difficult with the naming convention.
- A Unique Key which is shown at the last ALTER section is similar to a Primary Key as it ensures that all values in a column are different, meaning we will never have duplicates of the SalesPerson Column

In [6]:
USE RHNSCC_DD;
GO

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Products')
BEGIN
	CREATE TABLE dim.Products(
		  ProductID INT NOT NULL
		, Product NVARCHAR(50) NOT NULL
		, ProductCategory NVARCHAR(50) NOT NULL
		, ActiveFlag TINYINT NOT NULL
		);

ALTER TABLE dim.Products
ADD CONSTRAINT PK_Products PRIMARY KEY (ProductID)
END

GO

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'People')
BEGIN
	CREATE TABLE dim.People(
		pkPeople INT IDENTITY(1000,1) NOT NULL
	  , SalesPerson NVARCHAR(50) NOT NULL
	  , Team NVARCHAR(50) NOT NULL
	  , Province NVARCHAR(20) NOT NULL
	  , Phone NVARCHAR(10) NULL
	  )  ;

	ALTER TABLE dim.People
	ADD CONSTRAINT PK_People_LUP PRIMARY KEY(pkPeople);

	ALTER TABLE dim.People
    ADD CONSTRAINT UC_SalesPerson UNIQUE (SalesPerson);
END


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.031

### **2.** The final Code of this Query will be the creation of the **Fact Table: f.Sales**

Code Breakdown:

- Please refer to the above Dimensions Table Code Breakdown above up to the creation of the table.
- Adding Foreign Keys is very similar to adding Primary Keys however you are required to reference to the Primary Key of the table that the Foreign key has a relationship with. All other data remains constant.

In [7]:
USE RHNSCC_DD;
GO

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'Sales')
BEGIN
	CREATE TABLE f.Sales(
		  Product INT NOT NULL
		, SalesPerson INT NOT NULL
		, SalesDate DATE NOT NULL
		, TotalSales MONEY NOT NULL
		);

	ALTER TABLE f.Sales
	ADD CONSTRAINT FK_SALEStoPEOPLE FOREIGN KEY(SalesPerson)
	REFERENCES dim.People(pkPeople);

	ALTER TABLE f.Sales
	ADD CONSTRAINT FK_SALEStoPRODUCT FOREIGN KEY(Product)
	REFERENCES dim.Products(ProductID);

END

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009